# Qwen chat template preview for MATH datasets
This notebook loads a few rows from `train_MATH3-5.parquet` and `train_level3-5.parquet`,
applies the Qwen chat template the same way the trainer does (\`add_generation_prompt=True\`),
and shows the rendered text. Toggle `ADD_GENERATION_PROMPT` if you want to see the template
without the assistant prompt.


In [1]:
from pathlib import Path
import json
import pyarrow.parquet as pq
from transformers import AutoTokenizer
from IPython.display import Markdown, display

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
DATA_DIR = Path('data/MATH-500')


/data01/yunhochoi/miniconda3/envs/verl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data01/yunhochoi/miniconda3/envs/verl/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:
ADD_GENERATION_PROMPT = True  # set False to see template without assistant prefix

def load_rows(parquet_path: Path, n: int = 2):
    pf = pq.ParquetFile(parquet_path)
    rows = []
    it = pf.iter_batches(batch_size=1)
    for _ in range(min(n, pf.metadata.num_rows)):
        try:
            rows.append(next(it).to_pylist()[0])
        except StopIteration:
            break
    return rows

def render_template(messages, add_generation_prompt=None):
    if add_generation_prompt is None:
        add_generation_prompt = ADD_GENERATION_PROMPT
    return tokenizer.apply_chat_template(
        messages, add_generation_prompt=add_generation_prompt, tokenize=False
    )

from pathlib import Path

def show_examples(parquet_path: str | Path, n: int = 2, add_generation_prompt=None):
    parquet_path = Path(parquet_path) 
    rows = load_rows(parquet_path, n=n)
    for idx, row in enumerate(rows):
        messages = row["prompt"]
        rendered = render_template(messages, add_generation_prompt=add_generation_prompt)

        header = (
            f"### {parquet_path.name} | row {idx}\n"
            f"- data_source: {row.get('data_source')}\n"
            f"- reward_model: {row.get('reward_model')}\n"
            f"- extra_info keys: {sorted((row.get('extra_info') or {}).keys())}\n"
        )
        display(Markdown(header))
        display(Markdown("**messages (raw prompt list):**"))
        display(Markdown(f"```json\n{json.dumps(messages, indent=2)}\n```"))
        display(Markdown("**chat template output:**"))
        display(Markdown(f"```\n{rendered}\n```"))
        display(Markdown("---"))


In [11]:
show_examples("/data01/yunhochoi/verl/data/train_critique_3.1_4.parquet", n=2)


### train_critique_3.1_4.parquet | row 0
- data_source: critique_variants
- reward_model: {'ground_truth': '{"original_question": "What is the positive difference between $250\\\\%$ of 750 and $550\\\\%$ of 110?", "original_trajectory": "To find the positive difference, we first need to calculate 250% of 750 and 550% of 110. \\n\\nTo find 250% of 750, we multiply 750 by 2.5. \\n$250\\\\%$ of 750 = 750 * 2.5 = 1875\\n\\nTo find 550% of 110, we multiply 110 by 5.5. \\n$550\\\\%$ of 110 = 110 * 5.5 = 605\\n\\nThe positive difference is the absolute value of the difference between 1875 and 605. \\n$|1875 - 605| = 1270$", "variants": [{"snapshot": "Oct-2023", "question": "What is the positive difference between $250\\\\%$ of 750 and $550\\\\%$ of 110?", "answer": "1270"}, {"snapshot": "Nov-2023", "question": "What is the positive difference between $310\\\\%$ of 350 and $720\\\\%$ of 220?", "answer": "499"}, {"snapshot": "Dec-2023", "question": "What is the positive difference between $670\\\\%$ of 430 and $300\\\\%$ of 100?", "answer": "2581"}]}', 'style': 'rule'}
- extra_info keys: []


**messages (raw prompt list):**

```json
[
  {
    "content": "User Question: What is the positive difference between $250\\%$ of 750 and $550\\%$ of 110?\nCorrect Answer: 1270\n\nModel Solution Trace:\nTo find the positive difference, we first need to calculate 250% of 750 and 550% of 110. \n\nTo find 250% of 750, we multiply 750 by 2.5. \n$250\\%$ of 750 = 750 * 2.5 = 1875\n\nTo find 550% of 110, we multiply 110 by 5.5. \n$550\\%$ of 110 = 110 * 5.5 = 605\n\nThe positive difference is the absolute value of the difference between 1875 and 605. \n$|1875 - 605| = 1270$\n\nTask: Critique the solution trace above based on the correct answer. Identify specific logical errors or confirm the reasoning. Provide constructive feedback.",
    "role": "user"
  }
]
```

**chat template output:**

```
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
User Question: What is the positive difference between $250\%$ of 750 and $550\%$ of 110?
Correct Answer: 1270

Model Solution Trace:
To find the positive difference, we first need to calculate 250% of 750 and 550% of 110. 

To find 250% of 750, we multiply 750 by 2.5. 
$250\%$ of 750 = 750 * 2.5 = 1875

To find 550% of 110, we multiply 110 by 5.5. 
$550\%$ of 110 = 110 * 5.5 = 605

The positive difference is the absolute value of the difference between 1875 and 605. 
$|1875 - 605| = 1270$

Task: Critique the solution trace above based on the correct answer. Identify specific logical errors or confirm the reasoning. Provide constructive feedback.<|im_end|>
<|im_start|>assistant

```

...

**messages (raw prompt list):**

```json
[
  {
    "content": "User Question: What is the positive difference between $250\\%$ of 750 and $550\\%$ of 110?\nCorrect Answer: 1270\n\nModel Solution Trace:\nTo find the positive difference between $250\\%$ of 750 and $550\\%$ of 110, we need to calculate both values first.\n\n$250\\%$ of 750 is equal to $2.5 \\times 750 = 1875$.\n\n$550\\%$ of 110 is equal to $5.5 \\times 110 = 605$.\n\nNow, we need to find the positive difference between 1875 and 605. \n\nThe positive difference is $1875 - 605 = 1270$.\n\nThe final answer is 1270.\n\nTask: Critique the solution trace above based on the correct answer. Identify specific logical errors or confirm the reasoning. Provide constructive feedback.",
    "role": "user"
  }
]
```

**chat template output:**

```
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
User Question: What is the positive difference between $250\%$ of 750 and $550\%$ of 110?
Correct Answer: 1270

Model Solution Trace:
To find the positive difference between $250\%$ of 750 and $550\%$ of 110, we need to calculate both values first.

$250\%$ of 750 is equal to $2.5 \times 750 = 1875$.

$550\%$ of 110 is equal to $5.5 \times 110 = 605$.

Now, we need to find the positive difference between 1875 and 605. 

The positive difference is $1875 - 605 = 1270$.

The final answer is 1270.

Task: Critique the solution trace above based on the correct answer. Identify specific logical errors or confirm the reasoning. Provide constructive feedback.<|im_end|>
<|im_start|>assistant

```

---

In [10]:
show_examples("/data01/yunhochoi/verl/data/MATH-500/train_MATH3-5.parquet", n=2)


### train_MATH3-5.parquet | row 0
- data_source: HuggingFaceH4/MATH-500
- reward_model: {'ground_truth': '118', 'style': 'rule'}
- extra_info keys: ['index', 'split']


**messages (raw prompt list):**

```json
[
  {
    "content": "Please reason step by step, and put your final answer within \\boxed{}.",
    "role": "system"
  },
  {
    "content": "Let $a$ and $b$ be the two real values of $x$ for which\\[\\sqrt[3]{x} + \\sqrt[3]{20 - x} = 2\\]The smaller of the two values can be expressed as $p - \\sqrt{q}$, where $p$ and $q$ are integers. Compute $p + q$.",
    "role": "user"
  }
]
```

**chat template output:**

```
<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
Let $a$ and $b$ be the two real values of $x$ for which\[\sqrt[3]{x} + \sqrt[3]{20 - x} = 2\]The smaller of the two values can be expressed as $p - \sqrt{q}$, where $p$ and $q$ are integers. Compute $p + q$.<|im_end|>
<|im_start|>assistant

```

...

**messages (raw prompt list):**

```json
[
  {
    "content": "Please reason step by step, and put your final answer within \\boxed{}.",
    "role": "system"
  },
  {
    "content": "For how many integer values of $x$ is $5x^{2}+19x+16 > 20$ not satisfied?",
    "role": "user"
  }
]
```

**chat template output:**

```
<|im_start|>system
Please reason step by step, and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
For how many integer values of $x$ is $5x^{2}+19x+16 > 20$ not satisfied?<|im_end|>
<|im_start|>assistant

```

---